In [12]:
import skimage as ski
import numpy as np
import matplotlib.pyplot as plt
import math
import tensorflow as tf
import random

In [13]:
def generate__weight_matrix(size, scale = 1.0):

    stddev = scale/np.sqrt(np.prod(size))
    
    return np.random.normal(loc = 0, scale = stddev, size = size)

In [14]:
def max_pooling(img ,f = 2 ,s = 2):
   
    channels, height_img, weight_img = img.shape
    
    h = int((height_img - f)/s)+1 
    w = int((weight_img - f)/s)+1
    
    max_pool_output = np.zeros((channels, h, w)) 
    
    for i in range(channels):
        y = mpo_y = 0
    
        while y + f <= height_img:
            x = out_x = 0
    
            while x + f <= weight_img:
    
                max_pool_output[i, mpo_y, out_x] = np.max(img[i, y:y+f, x:x+f])
                x += s
                out_x += 1
    
            y += s
            mpo_y += 1
    
    return max_pool_output

In [15]:
def convolution(img, filters, bias, stride = 1,toprint = 0):
    
    if toprint == 1:
        print(img.shape)
        print(filt.shape)
    
    no_of_filters, filter_height, filter_width,filter_channel = filters.shape 
    img_height, img_width,img_channel = img.shape 
    
    output_dim = int((img_height - filter_height)/stride)+1
    
    conv_out = np.zeros((no_of_filters,output_dim,output_dim))
    

    for filt in range(no_of_filters):
        if toprint == 1:
            print("#######################################")
            print(filt)
        y = mpo_y = 0

        while y + filter_height <= img_height:
            x = out_x = 0
 
            while x + filter_height <= img_height:

                if toprint == 1:
                    print(filters[filt].shape)
                    print(img[y:y+filter_height, x:x+filter_height,:].shape)
                    print(y,y+filter_height, x,x+filter_height)
                    
                conv_out[filt, mpo_y, out_x] = (np.sum(filters[filt] * img[y:y+filter_height, x:x+filter_height,:]) + bias[filt])/(filter_height**2)
                x += stride
                out_x += 1
            
            y += stride
            mpo_y += 1
        
    return conv_out

In [16]:
def relu(cov_img):
    
    img_list = []
    
    for img in cov_img:
        
        img_list.append(np.maximum(img, 0))
    
    
    img_list = np.asarray(img_list)
    
#     print(img_list.shape)
    
#     for img in img_list:
        
#         ski.io.imshow(img)
#         ski.io.show()
        
    return img_list

In [17]:
def sigmoid(cov_img):
    
    img_list = []
    
    for img in cov_img:
        
        img_list.append(np.nan_to_num(1.0 / (1.0 + np.exp(-img))))
    
    
    img_list = np.asarray(img_list)
        
    return img_list

In [18]:
def tanh(cov_img):
    
    img_list = []
    
    for img in cov_img:
        
        img_list.append(np.nan_to_num(2.0/(1.0 + np.exp(-(2*img))) - 1))
    
    
    img_list = np.asarray(img_list)
        
    return img_list

In [19]:
def draw_img(img_list):

    for img in img_list:
        
        plt.imshow(img)
        plt.show()


In [20]:
def activation_function(arr,fuct):
    
    if fuct == "relu":
        
        return np.maximum(arr, 0)
    
    elif fuct == "sigmoid":
        
        return np.nan_to_num(1.0 / (1.0 + np.exp(-arr)))
    
    elif fuct == "tanh":
        
        return np.nan_to_num(2.0/(1.0 + np.exp(-(2*arr))) - 1)
    
    elif fuct == "softmax":
        
        e_x = np.exp(arr - np.max(arr))
        
        return e_x / e_x.sum(axis=0)

In [21]:
def train(img):
    
    l1_f = generate__weight_matrix([6, 5, 5, 3])

    l2_f = generate__weight_matrix([16, 5, 5, 6])
    
    l3_f = generate__weight_matrix([120, 5, 5, 16])

    b1_c = generate__weight_matrix([6])
    b2_c = generate__weight_matrix([16])
    b3_c = generate__weight_matrix([120])
        
#     print(l1_f)
#     print(l2_f)
    
    conv1 = convolution(img,l1_f,b1_c)
#     print(img.shape)
    print(conv1.shape)
    
#     draw_img(conv1)
    print("############################################################################")
    
    
    relu1 = relu(conv1)
#     draw_img(relu1)
    print(relu1.shape)
    print("############################################################################")
    
#     relu1 = sigmoid(conv1)
#     draw_img(relu1)
    
#     relu1 = tanh(conv1)
#     draw_img(relu1)

    max_pool1 = max_pooling(relu1).T
    print(max_pool1.shape)
#     draw_img(max_pool1.T)
    print("############################################################################")
    
    conv2 = convolution(max_pool1,l2_f,b2_c)
    print(conv2.shape)
#     draw_img(conv2)
    print("############################################################################")
    
    relu2 = relu(conv2)
    print(relu2.shape)
#     draw_img(relu2)
    print("############################################################################")

    max_pool2 = max_pooling(relu2).T
    print(max_pool2.shape)
#     draw_img(max_pool2.T)
    print("############################################################################")

    
    conv3 = convolution(max_pool2,l3_f,b3_c)
    print(conv3.shape)
#     draw_img(conv3)
    print("############################################################################")
    
    nn_input = conv3.flatten()
    print(nn_input.shape)

    mean, sd = np.mean(nn_input), np.std(nn_input)
    nn_input = (nn_input-mean)/sd

    
    outputNodes = 10
    
    
    W1 = generate__weight_matrix([nn_input.shape[0],84])
    W2 = generate__weight_matrix([84,10])
    
    b1 = generate__weight_matrix([84])
    b2 = generate__weight_matrix([10])
    
    A_H1 = activation_function((np.dot(W1.T,nn_input) + b1),"sigmoid")
    A_OUT = activation_function((np.dot(W2.T,A_H1) + b2),"softmax")
    print(np.asmatrix(A_OUT).T)
    print("Prediction: ",np.argmax(A_OUT))

In [22]:
img = ski.io.imread("2.jpeg")
train(img)

(6, 28, 28)
############################################################################
(6, 28, 28)
############################################################################
(14, 14, 6)
############################################################################
(16, 10, 10)
############################################################################
(16, 10, 10)
############################################################################
(5, 5, 16)
############################################################################
(120, 1, 1)
############################################################################
(120,)
[[0.09270989]
 [0.15174947]
 [0.07390006]
 [0.16907015]
 [0.09248922]
 [0.09661555]
 [0.06664278]
 [0.11173337]
 [0.07345948]
 [0.07163004]]
Prediction:  3
